Following Teemu's presentation earlier this week, the trained FastText classifier was downloaded using the following lines:
<code>
    ft_classifier.quantize(input=input, retrain=True)
    ft_classifier.save_model("ft_classifier.ftz")
</code>

In [21]:
import fasttext
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
# import libvoikko #perusmutoistamiseen käytetty kirjasto
import spacy

In [22]:
nlp = spacy.load("fi_core_news_sm")

In [23]:
# !python -m spacy download fi_core_news_sm

In [24]:

print(stopwords.words('finnish'))

['olla', 'olen', 'olet', 'on', 'olemme', 'olette', 'ovat', 'ole', 'oli', 'olisi', 'olisit', 'olisin', 'olisimme', 'olisitte', 'olisivat', 'olit', 'olin', 'olimme', 'olitte', 'olivat', 'ollut', 'olleet', 'en', 'et', 'ei', 'emme', 'ette', 'eivät', 'minä', 'minun', 'minut', 'minua', 'minussa', 'minusta', 'minuun', 'minulla', 'minulta', 'minulle', 'sinä', 'sinun', 'sinut', 'sinua', 'sinussa', 'sinusta', 'sinuun', 'sinulla', 'sinulta', 'sinulle', 'hän', 'hänen', 'hänet', 'häntä', 'hänessä', 'hänestä', 'häneen', 'hänellä', 'häneltä', 'hänelle', 'me', 'meidän', 'meidät', 'meitä', 'meissä', 'meistä', 'meihin', 'meillä', 'meiltä', 'meille', 'te', 'teidän', 'teidät', 'teitä', 'teissä', 'teistä', 'teihin', 'teillä', 'teiltä', 'teille', 'he', 'heidän', 'heidät', 'heitä', 'heissä', 'heistä', 'heihin', 'heillä', 'heiltä', 'heille', 'tämä', 'tämän', 'tätä', 'tässä', 'tästä', 'tähän', 'tallä', 'tältä', 'tälle', 'tänä', 'täksi', 'tuo', 'tuon', 'tuotä', 'tuossa', 'tuosta', 'tuohon', 'tuolla', 'tuolta', 

In [26]:
# Load tweets datasets
# Note: May need to install openpyxl library for the following line to work
# pip install openpyxl
df = pd.read_excel('shared_data/sample_texts.xlsx')


In [27]:
df

,Unnamed: 0,text_nousernames,author_username,Name,Position,eduskunta-vkk,twitter_handle
0,0,RT : Credit where due. Biden warned of the Ru...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
1,1,RT : I look forward to welcoming Sweden and F...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
2,2,"RT : ""Hätten wir damals wegen der Krim stärke...",TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
3,3,RT : My deepest condolences for the victims a...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
4,4,RT : Thank you Kamala Harris and \n for an...,TyttiTup,Tytti Tuppurainen,Eurooppa- ja omistajaohjausministeri,"eurooppa-, kulttuuri- ja urheiluministeri",TyttiTup
...,...,...,...,...,...,...,...
1163,1163,RT : Facebookissa K-studio https://t.co/qFL2P...,MikaLintila,Mika Lintilä,Elinkeinoministeri,elinkeinoministeri,MikaLintila
1164,1164,RT : Ministeri Mika Lintilä antoi itselleen a...,MikaLintila,Mika Lintilä,Elinkeinoministeri,elinkeinoministeri,MikaLintila
1165,1165,Kiitokset Jorma asiantuntemuksestasi Suomen ...,MikaLintila,Mika Lintilä,Elinkeinoministeri,elinkeinoministeri,MikaLintila
1166,1166,emme ole kirjeessä mukana koska kannattama...,MikaLintila,Mika Lintilä,Elinkeinoministeri,elinkeinoministeri,MikaLintila


In [46]:
tweets = df[['text_nousernames', 'eduskunta-vkk']]

In [47]:
tweets

,text_nousernames,eduskunta-vkk
0,RT : Credit where due. Biden warned of the Ru...,"eurooppa-, kulttuuri- ja urheiluministeri"
1,RT : I look forward to welcoming Sweden and F...,"eurooppa-, kulttuuri- ja urheiluministeri"
2,"RT : ""Hätten wir damals wegen der Krim stärke...","eurooppa-, kulttuuri- ja urheiluministeri"
3,RT : My deepest condolences for the victims a...,"eurooppa-, kulttuuri- ja urheiluministeri"
4,RT : Thank you Kamala Harris and \n for an...,"eurooppa-, kulttuuri- ja urheiluministeri"
...,...,...
1163,RT : Facebookissa K-studio https://t.co/qFL2P...,elinkeinoministeri
1164,RT : Ministeri Mika Lintilä antoi itselleen a...,elinkeinoministeri
1165,Kiitokset Jorma asiantuntemuksestasi Suomen ...,elinkeinoministeri
1166,emme ole kirjeessä mukana koska kannattama...,elinkeinoministeri


In [48]:
# Methods imported from the original Notebook
def preprocess_word(word, stopwords):
    word = re.sub("[^A-Za-z0-9ÄäÖö-]+", "", word)
    word = re.sub("-", " ", word)
    word = word.strip().lower()
    if len(word) < 2:
        return ""
    if type(word) != None:
        if word in stopwords:
            return ""
        return word

def preprocess_sent(sent, stopwords):
    if sent != "":
        # lemmatize words using spacy
        doc = nlp(sent)
        sent = " ".join([token.lemma_ for token in doc])
        
        words = sent.strip().split(" ")
        final_sent = " ".join([preprocess_word(word, stopwords) for word in words if type(word) != None])
        return final_sent

def preprocess_row(teksti, stopwords):
    if type(teksti) == str:
        sents = teksti.strip().split(".")
        sents = filter(None,sents)
        teksti = ".".join([preprocess_sent(sent, stopwords) for sent in sents if type(sent) != None])
        return teksti

In [49]:
tweets

,text_nousernames,eduskunta-vkk
0,RT : Credit where due. Biden warned of the Ru...,"eurooppa-, kulttuuri- ja urheiluministeri"
1,RT : I look forward to welcoming Sweden and F...,"eurooppa-, kulttuuri- ja urheiluministeri"
2,"RT : ""Hätten wir damals wegen der Krim stärke...","eurooppa-, kulttuuri- ja urheiluministeri"
3,RT : My deepest condolences for the victims a...,"eurooppa-, kulttuuri- ja urheiluministeri"
4,RT : Thank you Kamala Harris and \n for an...,"eurooppa-, kulttuuri- ja urheiluministeri"
...,...,...
1163,RT : Facebookissa K-studio https://t.co/qFL2P...,elinkeinoministeri
1164,RT : Ministeri Mika Lintilä antoi itselleen a...,elinkeinoministeri
1165,Kiitokset Jorma asiantuntemuksestasi Suomen ...,elinkeinoministeri
1166,emme ole kirjeessä mukana koska kannattama...,elinkeinoministeri


In [68]:
#Apufunktio, jolla voidaan luoda fastText-yhteensopiva opetus- ja tekstitiedosto

def preprocess_ft_df(df):
    df["label"] = ["__label__" + "_".join(str(x).split(" ")) for x in df["eduskunta-vkk"].values]
    df["preprocessed"] = [preprocess_row(sentence, stopwords.words('finnish')) for sentence in df["text_nousernames"]]
    df = df.drop(["eduskunta-vkk", "text_nousernames"], axis = 1)
    return df

In [69]:
new_df = preprocess_ft_df(tweets)

/var/folders/s7/s_ybn2ps78q6ycl36wt3sv814z5bsf/T/ipykernel_63259/2270654315.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["label"] = ["__label__" + "_".join(str(x).split(" ")) for x in df["eduskunta-vkk"].values]
/var/folders/s7/s_ybn2ps78q6ycl36wt3sv814z5bsf/T/ipykernel_63259/2270654315.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df["preprocessed"] = [preprocess_row(sentence, stopwords.words('finnish')) for sentence in df["text_nousernames"]]


In [70]:
new_df

,label,preprocessed
0,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt credit where due.biti warned of the russ...
1,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt look forward to welcoming sweden and fi...
2,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt hätten wir damals wegen der krim stärke...
3,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt my deepest condolences for the victims a...
4,"__label__eurooppa-,_kulttuuri-_ja_urheiluminis...",rt thank you kamala harris and for an...
...,...,...
1163,__label__elinkeinoministeri,rt facebook k studio httpst.co qfl2pj4t99
1164,__label__elinkeinoministeri,rt ministeri mika lintilä antaa arvosata k...
1165,__label__elinkeinoministeri,kiitokset jorma asiantuntemuksestata suomi ene...
1166,__label__elinkeinoministeri,kirje mukana kannattamtaa ydinvoima lisäksi...


In [83]:

model = fasttext.load_model("model_filename.ftz")

In [106]:
predictions = [p[0] for p in predictions[0]]

In [106]:
predictions = [p[0] for p in predictions[0]]

In [111]:
hits = 0
miss = 0
for pred, truth in zip(predictions, new_df['label'].values):
    if pred==truth:
        hits += 1
    else:
        miss += 1

In [113]:
hits, miss

(180, 988)

In [114]:
hits/miss

0.18218623481781376

In [116]:
len(new_df['label'].unique())

16